TODOS

1. convert mol dictionary to class variable
2. add widgets and dynamicism/plots
3. add more explanation/text describing functions
4. add counter to avoid overwriting electronic structure input/output


#### Import necessary headers

In [1]:
import sys
print(sys.path, sys.executable)
print(sys.version)

['', '/home/oosode/.conda/envs/python36/lib/python36.zip', '/home/oosode/.conda/envs/python36/lib/python3.6', '/home/oosode/.conda/envs/python36/lib/python3.6/lib-dynload', '/home/oosode/.conda/envs/python36/lib/python3.6/site-packages', '/home/oosode/.conda/envs/python36/lib/python3.6/site-packages/IPython/extensions', '/home/oosode/.ipython'] /home/oosode/.conda/envs/python36/bin/python
3.6.5 |Anaconda, Inc.| (default, Apr 29 2018, 16:14:56) 
[GCC 7.2.0]


In [3]:
import os
import sys

import ipywidgets as widgets
from subprocess import Popen, PIPE
from parsl import *
from math import *

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import cirpy


In [4]:
a=cirpy.resolve('water', 'xyz')
print(a)

#b=cirpy.resolve('ice','image_url')
%matplotlib inline

#set plot sizes
plt.rcParams['figure.figsize'] = (10, 2)  # (width, height)
plt.rcParams['font.size'] = 20
plt.rcParams['legend.fontsize'] = 16

3
H2O
O   -0.0000   -0.0589   -0.0000
H   -0.8110    0.4677    0.0000
H    0.8110    0.4677    0.0000



Widget test

In [5]:
widgets.RadioButtons(
    options=['MOLPRO', 'NWChem'],
    description='Program:',
    disabled=False
)

RadioButtons(description='Program:', options=('MOLPRO', 'NWChem'), value='MOLPRO')

In [ ]:
#import moldesign as mdt
#mol = mdt.from_name('ethylene')
#mol.draw()

In [ ]:
#from ase import Atoms
#from ase.visualize import view
#d = 1.1
#co = Atoms('CO', positions=[(0, 0, 0), (0, 0, d)])
#view(co, viewer='VMD')

### Function to write electronic structure input files

In [ ]:
def write_input(ID,mol,theory,basis,program,optimization,f12,cp=False):
    
    natoms=int(mol['natoms'])
    
    if program=="MOLPRO":
        
        fout=open("%s.%06d.com"%("molpro",ID),"w")
        fout.write("***, %s %s \n"%(mol['name'],optimization))
        fout.write("memory,200,M\n")
        fout.write("geometry={\n")
        
        for i in range(natoms):

            fout.write("%s%d "%(mol['atom'][i],i))
            fout.write("%15.10f, %15.10f, %15.10f"%(mol['coord'][i][0],\
                                                    mol['coord'][i][1],\
                                                    mol['coord'][i][2]))
            if i==natoms-1:
                fout.write("}\n")
            else:
                fout.write("\n")

        fout.write("basis=%s"%(basis))
        fout.write("\n")

        if cp:
            fout.write("text,cp calculation for %s %dmer_%i\n"%(name,m,n))
            fout.write("dummy,")
            for i in range(atoms):
                if molid[i] in dum:
                    tmp=index*atoms
                    fout.write("%s%d,"%(names[tmp+i],i))
                    fout.write("\n")
                else:
                    fout.write("text,calculation for %s %dmer\n"%(name,m))

        if theory!="hf":
            fout.write("hf\n")
            fout.write("%s"%(theory))
            if f12:
                fout.write("-f12\n")
            else:
                fout.write("\n")
        else:
            fout.write("hf\n")
            
        if optimization=="Minimum":
            fout.write("optg\n")
            fout.write("maxit=50\n")
        if optimization=="Saddle":
            fout.write("optg")

        fout.write("e_mer=energy\n")
        if (f12):
            fout.write("e_mer_F12a=energy(1)\n")
            fout.write("e_mer_F12b=energy(2)\n")
        fout.write("\n")
        fout.close()
        return
        
    if program=="NWChem":
        pass

### Function to run electronic structure program

In [ ]:
def run_input(ID,program):
    
    if program=="MOLPRO":
        process = Popen(["/Users/oosode/Software/molpro-2015/bin/molpro.exe", "molpro.%06d.com"%(ID)], stdout=PIPE)
    elif program=="NWChem":
        process = Popen([],stdout=PIPE)
    else:
        raise IOError("Unknown program.") 
        
    return

In [38]:
from parsl.configs.local import localThreads

config = {
    "sites": [
        {
            "site": "midway_ipp_multicore",
            "auth": {
                "channel": "local",
                "hostname": "swift.rcc.uchicago.edu",
                "scriptDir": "/scratch/midway2/{0}/parsl_scripts".format('oosode')
            },
            "execution": {
                "executor": "ipp", # remote executor
                "provider": "slurm", 
                "block": {
                    "nodes": 1, # nodes per block
                    "taskBlocks": 1, # tasks per block
                    #"taskBlocks":20, 
                    "walltime": "00:05:00",
                    "initBlocks": 1,
                    "minBlocks" : 1,
                    "maxBlocks": 100,
                    "options": {'partition' : 'westmere',
                                'overrides' : 'source activate python36' }
                }
            }
        }, 
        {
            "site": "midway_ipp_singlenode",
            "auth": {
                "channel": "local",
                "hostname": "swift.rcc.uchicago.edu",
                "scriptDir": "/scratch/midway2/{0}/parsl_scripts".format('oosode')
            },
            "execution": {
                "executor": "ipp", # remote executor
                "provider": "slurm", 
                "block": {
                    "nodes": 1, # nodes per block
                    "taskBlocks": 1, # tasks per block
                    #"taskBlocks":20, 
                    "walltime": "00:05:00",
                    "initBlocks": 1,
                    "minBlocks" : 1,
                    "maxBlocks": 1,
                    "options": {'partition' : 'westmere',
                                'overrides' : 'source activate python36' }
                }
            }
        }
        
    ],
    "globals": {
        "lazyErrors": True,
    },
    #"controller" : {'publicIp' : ''}
}

dfk = DataFlowKernel(config=config)
#dfk = DataFlowKernel(config=localThreads)

Caught exception : Command 'squeue --job 46289462' timed out after 10 seconds
Caught exception : Command 'squeue --job 46289271' timed out after 10 seconds


In [32]:

@App('bash', dfk, sites=['midway_ipp_multicore'])
def molpro(ID, outputs=[], stderr=None, stdout=None):
    return "uname -a; /home/oosode/software/molpro-2015/bin/molpro.exe  molpro.{0:06d}.com; touch molpro.{0:06d}.xml molpro.{0:06d}.out; sleep 15".format(ID)

for ID in range(100):
    fut = molpro(ID, outputs=["molpro.%06d.out"%(ID), "molpro.%06d.xml"%(ID)],
                 stdout="molpro.%06d.stdout"%(ID),
                 stderr="molpro.%06d.stderr"%(ID))

In [35]:
print(fut)

<AppFuture at 0x7f8db902b828 state=finished returned int>


In [40]:
!cat molpro.000002.stderr

/bin/bash: /Users/oosode/Software/molpro-2015/bin/molpro.exe: No such file or directory
Caught exception : Command 'squeue --job 46289271' timed out after 10 seconds
Caught exception : Command 'squeue --job 46289462' timed out after 10 seconds


In [39]:
!qstat -u $USER

### Function to read electronic structure ouptut

In [ ]:
def read_output(ID,program,mol):
    
    energy={}
    
    if program=="MOLPRO":    
        c=False
        fout="molpro.%06d.out"%(ID)
        if not os.path.isfile(fout):
            raiseIOError("No file for molpro.%06d.com"%(ID))

        else:
            fl=open(fout).readlines()

            for l in range(len(fl)-1,len(fl)-5,-1):
                if fl[l].find("Variable memory released")>=0:
                    c=True
                    break

            if (not c):
                energy[0]="inf"
                print("No energies for file molpro.%06d.com"%(ID))
                return

            else:
                for l,line in enumerate(fl):
                    if line.find("SETTING E_MER ")>=0: # energy
                        energy['simple']=float(line.strip().split()[-2].replace("D","E"))
                        mol['energy']=energy
                        
                    #if line.find("SETTING E_MER_F12A ")>=0: # F12a energy
                    #    energy['f12a']=float(line.strip().split()[-2].replace("D","E"))
                    #    mol['energy']=energy
                    #elif line.find("SETTING E_MER_F12B ")>=0: # F12b energy
                    #    energy['f12b']=float(line.strip().split()[-2].replace("D","E"))
                    #    mol['energy']=energy
                    
                    elif line.find("END OF GEOMETRY OPTIMIZATION.")>=0:
                        natoms=int(fl[l+4].split()[0])
                        for n in range(natoms):
                            for i in range(3):
                                mol['coord'][n][i]=float(fl[l+6+n].split()[i+1])
                    
            return



### Choose electronic structure program

In [ ]:
w=widgets.RadioButtons(
    options=['MOLPRO', 'NWChem'],
    description='Program:',
    disabled=False
)
display(w)
#program=w.value

In [ ]:
print(w.value)
program='MOLPRO'

### Choose level of theory and basis set

In [ ]:
wwidgets.Dropdown(
    options=['HF', 'MP2', 'CCSD(T)'],
    value='HF',
    description='Level of theory:',
    disabled=False,
)

widgets.RadioButtons(
    options=['True', 'False'],
    description='Explicit correlation:',
    disabled=False
)
widgets.Dropdown(
    options=['aug-cc-pVDZ', 'aug-cc-pVTZ', 'aug-cc-pVQZ',\
             'cc-pVDZ-F12', 'cc-pVTZ-F12', 'cc-pVQZ-F12'],
    value='aug-cc-pVDZ',
    description='Basis set:',
    disabled=False,
)

In [ ]:
theory="mp2"
basis="aug-cc-pVDZ"
f12=True

There should eventually be a Query function that lets you pick a molecule perhaps from the CCCBDB at a certain level of theory and use it to begin the calculations.

### Choose Molecule 1

In [ ]:
# this should be a dictionary
mol1 = {'name':"CO2", \
        'coord':[[ 0.000, 0.000, 0.000], \
               [ 1.162, 0.000, 0.000], \
               [-1.162, 0.000, 0.000]], \
        'mass':[ 12.0000, 16.00000, 16.0000], \
        'natoms':3, \
        'atom':["C","O","O"]}
                           
#mol1 =  [["C",[ 0.000, 0.000, 0.000],12.0000],\
#         ["O",[ 1.162, 0.000, 0.000],12.0000],\
#         ["O",[-1.162, 0.000, 0.000],12.0000]]]

### Choose Molecule 2

In [ ]:
mol2 = {'name':"CO2", \
        'coord':[[ 0.000, 0.000, 0.000], \
               [ 1.162, 0.000, 0.000], \
               [-1.162, 0.000, 0.000]], \
        'mass':[ 12.0000, 16.00000, 16.0000], \
        'natoms':3, \
        'atom':["C","O","O"]}

In [ ]:
pair=[mol1,mol2]

### Optimize monomers

In [ ]:
for i in range(2): # loop over monomers
    #write_input(i,pair[i],theory,basis,program,"Minimum",f12,False)
    #run_input(i,program)
    read_output(i,program,pair[i])
    print(pair[i]['coord'],print(pair[i]['energy']))

## Dimer initiation

Molecule 1 is always placed at the origin (0,0,0) oriented in space with $\theta_1$ and $\phi_1$. The range of $\theta_1$ is [0°, 180°] and the range of $\phi_1$ is [0, 360°). Molecule 2 is placed using spherical coordinates at some distance R from the origin, with $\theta_d$ and $\phi_d$ ranging from [0°, 180°] and [0, 360°), respectively. The orientation of Molecule 2 is denoted by $\theta_2$ and $\phi_2$ values. All angles are in degree units and distances are in angstroms.

<img src='https://upload.wikimedia.org/wikipedia/commons/4/4f/3D_Spherical.svg' style="width: 400px;">

#### Select orientations

In [ ]:
# orient molecule 0
theta_0 = 0
phi_0 = 0

# orient molecule 1
theta_1 = 180
phi_1 = 180

# position
R = 4.0
theta_d = 0
phi_d = 0

In [ ]:
def center_of_mass(mol):
    
    x=0.0
    y=0.0
    z=0.0
    
    mass=mol["mass"]
    totalmass=0.0
    
    for a,atom in enumerate(mol["coord"]):
        
        x+=atom[0]*mass[a]
        y+=atom[1]*mass[a]
        z+=atom[2]*mass[a]
        totalmass+=mass[a]
    
    x=x/totalmass
    y=y/totalmass
    z=z/totalmass
    
    mol["com"]=[x,y,z]
    return

def center_molecule(mol):
    
    com=mol["com"]
    
    for atom in mol["coord"]:
        
        atom[0]-=com[0]
        atom[1]-=com[1]
        atom[2]-=com[2]
        
    return


def rotate_molecule(mol,theta,phi):
    
    for atom in mol["coord"]:
        
        x=atom[0]
        y=atom[1]
        z=atom[2]

        r=sqrt(x*x + y*y + z*z)
        if r<1.0e-10:
            continue
        t=acos(z/r)
        p=atan2(y,x)
        
        t+=radians(theta)
        p+=radians(phi)
        
        atom[0]=r*sin(t)*cos(p)
        atom[1]=r*sin(t)*sin(p)
        atom[2]=r*cos(t)
    
    return

def position_molecule(mol,R,theta,phi):
    
    x=R*sin(theta)*cos(phi)
    y=R*sin(theta)*sin(phi)
    z=R*cos(theta)
    
    for atom in mol["coord"]:
        
        atom[0]+=x
        atom[1]+=y
        atom[2]+=z
        
    return
        

#### Orient dimer

In [ ]:
center_of_mass(pair[0])
center_of_mass(pair[1])

center_molecule(pair[0])
center_molecule(pair[1])

rotate_molecule(pair[0],theta_0,phi_0)
rotate_molecule(pair[1],theta_1,phi_1)

position_molecule(pair[1],R,theta_d,phi_d)

In [ ]:
dim  = {'name':"CO2", \
        'coord':pair[0]['coord'] + pair[1]['coord'], \
        'mass':pair[0]['mass'] + pair[1]['mass'], \
        'natoms':pair[0]['natoms'] + pair[1]['natoms'], \
        'atom':pair[0]['atom'] + pair[1]['atom'],\
        'frag':[0]*pair[0]['natoms'] + [1]*pair[1]['natoms'],\
        'natoms1':pair[0]['natoms'],'natoms2':pair[0]['natoms']}

#print(dim)

### Find minimimum, saddle point or leave alone

In [ ]:
w=widgets.RadioButtons(
    options=['Minimum', 'Saddle', 'None'],
    description='Optimization:',
    disabled=False
)
#display(w)
optimization="Minimum"

In [ ]:
if optimization=="None":
    pass
elif optimization=="Minimum" or "Saddle":
    #write_input(3,dim,theory,basis,program,optimization,f12,False)
    #run_input(3,program)
    #read_output(3,program)
    
else:
    raise IOError("Unknown optimization.")

In [ ]:
w=widgets.FloatRangeSlider(
    value=[5, 7.5],
    min=1,
    max=10.0,
    step=0.1,
    description='Intermolecular distance:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
#w.value
#display(w)    

In [ ]:
minimum=2.0
maximum=8.0
increment=0.1

ranges=[]
x=minimum
while x<=maximum:
    ranges.append(x)
    x+=increment
print(ranges)

In [ ]:
def pes_scan(dim,ranges):

    center_of_mass(dim[0])
    center_of_mass(dim[1])

    center_molecule(dim[0])
    center_molecule(dim[1])
    
    com1=dim[0]["com"]
    com2=dim[1]["com"]
    
    # get distance vector
    v=[com2[0]-com1[0],\
       com2[1]-com1[1],\
       com2[2]-com1[2]]

    normsq = 0.0
    for l in range(3):
        normsq += v[l]*v[l]

    norm = math.sqrt(normsq)
    for l in range(3):
        v[l] /= norm
        
    # center monomer 2
    for j in range(natoms):
        if dim["frag"][j]==1: 
            dimer_scans[i]["coord"][j][0]-=com2[0]
            dimer_scans[i]["coord"][j][1]-=com2[1]
            dimer_scans[i]["coord"][j][2]-=com2[2]
            
    self.coord=[]

    # do pes scan
    dimer_scans=[]
    natoms=dim["natoms"]
    
    for i,r in enumerate(ranges):

        dimer_scans.append(dim)

        for j in range(natoms):
            if dim["frag"][j]==1:
                dimer_scans[i]["coord"][j][0]+=v[0]*r
                dimer_scans[i]["coord"][j][1]+=v[1]*r
                dimer_scans[i]["coord"][j][2]+=v[2]*r
           
    return

In [ ]:
pes_scans()

for i in range(ranges): # loop over monomers
    write_input(i,pair[i],theory,basis,program,"Minimum",f12,False)

    


In [ ]:

for i in range(ranges): # loop over monomers 
    run_input(i,program)

In [ ]:
for i in range(ranges): # loop over monomers
    read_output(i,program,dimer_scans[i])
    print(dimer[i]['coord'],print(dimer[i]['energy']))

Test

In [ ]:
def f(x):
    return x

In [ ]:
interact(f, x=10);